<a href="https://colab.research.google.com/github/Armada99/Link-Checker/blob/main/Link_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import threading
import html
from bs4 import BeautifulSoup
import requests

url = "https://reddit.com/r/FREEMEDIAHECKYEAH/wiki/"
names = [
            "adblock-vpn-privacy",
            "video",
            "audio",
            "games",
            "reading",
            "android",
            "download",
            "torrent",
            "edu",
            "tools-misc",
            "misc",
            "non-eng",
            "storage",
            "img-tools"
        ]

with open("links.txt", "w") as f:
    f.write("")

def get_url(n):
    finalurl = url + n + ".json"
    resp = requests.get(
            finalurl, 
            headers = {"user-agent": "fmhy"}
            )
    r = resp.json()
    soup = BeautifulSoup(html.unescape(r["data"]["content_html"]), 'html.parser')

    print("saving links from " + n + " to links.txt")
    with open("links.txt", "a") as f:
        for link in soup.find_all("a"):
            if "http" in link.get("href"):
                f.write(link.get("href") + "\n")

threads = []
for name in names:
    locals()[name] = threading.Thread(target=get_url, args=(name,))
    threads.append(locals()[name])

for thread in threads:
    thread.start()

In [6]:
import requests
import threading

with open("links.txt", "r") as f:
    links = list(f)
links_clean = set(links)
links = list(links_clean)
with open("status.txt", "w") as f:
    f.write("")

Urls = []
def link_checker(link):
    try:
        r = requests.get(link.strip(), timeout=1.0)
        Urls.append(f"{r.status_code if r.status_code != 200 else ''} {r.url}\n")
    except Exception as e:
        Urls.append(f"error {link.strip()}\n")
        pass


        
    
threads = []
for i, link in enumerate(links):
    threads.append(threading.Thread(target=link_checker, args=(link,)))


for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

with open("status.txt", "a") as f:
    for url in Urls:
        f.write(url)

print("Done.")